# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

We have created a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks = stocks[~stocks['Ticker'].isin(['DISCA', 'HFC','VIAC','WLTW'])]
stocks.describe()

,Ticker
count,501
unique,501
top,A
freq,1


## Acquiring an API Token
Now we have imported our IEX Cloud API token. This is the data provider that we have used.

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 74389397, 'calculationPrice': 'close', 'change': 1.02, 'changePercent': 0.00683, 'close': 0, 'closeSource': 'afiiofcl', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': None, 'highTime': None, 'iexAskPrice': None, 'iexAskSize': None, 'iexBidPrice': None, 'iexBidSize': None, 'iexClose': 148.84, 'iexCloseTime': 1739172950068, 'iexLastUpdated': None, 'iexMarketPercent': None, 'iexOpen': 156.06, 'iexOpenTime': 1725959583279, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexVolume': None, 'lastTradeTime': 1699897913564, 'latestPrice': 152.4, 'latestSource': 'Close', 'latestTime': 'December 13, 2022', 'latestUpdate': 1677809632759, 'latestVolume': None, 'low': 0, 'lowSource': None, 'lowTime': None, 'marketCap': 2405810071336, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTim

## Parsing Our API Call

In [5]:
price = data['latestPrice']
market_cap = data['marketCap']

## Adding Our Stocks Data to a Pandas DataFrame

In [6]:
my_columns = [ 'Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [7]:
# final_dataframe = final_dataframe.append(
 
#         pd.Series(
#             [
#                 symbol,
#                 price,
#                 market_cap,
#                 'N/A'
#             ],
#             index = my_columns
#         ),
#         ignore_index=True
 
# )

# final_dataframe

df1 = pd.DataFrame([[symbol, price, market_cap, 'N/A']], columns = my_columns)

new_dataframe = pd.concat([final_dataframe,df1])

new_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,152.4,2405810071336,N/A


## Looping Through The Tickers in Our List of Stocks

In [8]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker'][:5]:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
#     final_dataframe = final_dataframe.append(
#         pd.Series(
#         [
#             stock,
#             data['latestPrice'],
#             data['marketCap'],
#             'N/A'
#         ],
#         index = my_columns),
#     ignore_index = True
#     )
    df2 = pd.DataFrame([[stock, data['latestPrice'], data['marketCap'], 'N/A']], columns = my_columns)
    final_dataframe = pd.concat([final_dataframe,df2])

In [10]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,149.83,2388012872340,N/A
0,AAL,147.42,2380247815047,N/A
0,AAP,147.15,2381102653638,N/A
0,AAPL,146.80,2429320898185,N/A
0,ABBV,145.72,2427341983944,N/A


In [11]:
final_dataframe = final_dataframe.reset_index(drop='True')
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,149.83,2388012872340,N/A
1,AAL,147.42,2380247815047,N/A
2,AAP,147.15,2381102653638,N/A
3,AAPL,146.80,2429320898185,N/A
4,ABBV,145.72,2427341983944,N/A


## Using Batch API Calls to Improve Performance

In [12]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [13]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
#         final_dataframe = final_dataframe.append(
#                                         pd.Series([symbol, 
#                                                    data[symbol]['quote']['latestPrice'], 
#                                                    data[symbol]['quote']['marketCap'], 
#                                                    'N/A'], 
#                                                   index = my_columns), 
#                                         ignore_index = True)
          df3 = pd.DataFrame([[symbol, 
                               data[symbol]['quote']['latestPrice'], 
                               data[symbol]['quote']['marketCap'], 
                               'N/A']],
                             columns = my_columns)
          final_dataframe = pd.concat([final_dataframe,df3])
        
    
final_dataframe    

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,161,47804208193,N/A
0,AAL,13.62,9159308854,N/A
0,AAP,152.34,9029933640,N/A
0,AAPL,151.27,2342739277410,N/A
0,ABBV,171.56,293125667456,N/A
...,...,...,...,...
0,YUM,130.96,37229359355,N/A
0,ZBH,130.66,27900028373,N/A
0,ZBRA,265.86,14118962263,N/A
0,ZION,48.88,7376812036,N/A


In [14]:
final_dataframe = final_dataframe.reset_index(drop='True')
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,161,47804208193,N/A
1,AAL,13.62,9159308854,N/A
2,AAP,152.34,9029933640,N/A
3,AAPL,151.27,2342739277410,N/A
4,ABBV,171.56,293125667456,N/A
...,...,...,...,...
496,YUM,130.96,37229359355,N/A
497,ZBH,130.66,27900028373,N/A
498,ZBRA,265.86,14118962263,N/A
499,ZION,48.88,7376812036,N/A


## Calculating the Number of Shares to Buy

In [15]:
portfolio_size = input('Enter the value of your portfolio:')

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \nPlease try again:")
    portfolio_size = input('Enter the value of your portfolio:')
    val = float(portfolio_size)

Enter the value of your portfolio:10000000


In [16]:
position_size = val/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])

final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,161,47804208193,123
1,AAL,13.62,9159308854,1465
2,AAP,152.34,9029933640,131
3,AAPL,151.27,2342739277410,131
4,ABBV,171.56,293125667456,116
...,...,...,...,...
496,YUM,130.96,37229359355,152
497,ZBH,130.66,27900028373,152
498,ZBRA,265.86,14118962263,75
499,ZION,48.88,7376812036,408


## Formatting Our Excel Output
We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

### Initializing our XlsxWriter Object

In [20]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

## Creating the Formats We'll Need For Our .xlsx File
Formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* $XX.XX format for stock prices

- $XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [21]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )


## Applying the Formats to the Columns of Our .xlsx File

In [22]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

## Saving Our Excel Output


In [23]:
writer.save()